## Image Generation

In [ ]:
# !pip install transformers==4.49 diffusers==0.32.2
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/utils.py
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/LandscapeGenerator.py

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = ""

from datasets import load_dataset
from PIL import Image as PImage

from LandscapeGenerator import LandscapeGenerator

In [ ]:
newsdata_ds = load_dataset("thiagohersan/newsdata-images", split="newsdata")
newsdata_data = [img for img in newsdata_ds]
newsdata_data_date = sorted(newsdata_data, key=lambda x: x["pubDate"], reverse=True)[:32]

In [ ]:
models = [
  "runwayml/stable-diffusion-inpainting",
  "stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting"
]

mLG = LandscapeGenerator(newsdata_data_date, models[1])

### New landscape

In [ ]:
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=8, label="test00")

### Continue with seed

In [ ]:
seed = PImage.open("./imgs/test01/test01_07.jpg")
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=8, label="test02", seed_img=seed)

### Graft

In [ ]:
limg = PImage.open("./imgs/test02/test01_01.jpg")
rimg = PImage.open("./imgs/test02/test01_02.jpg")
display(rimg)

prompt,img_in,mask_in = mLG.prep_graft(limg, rimg, keep_width=256, right_offset=20)
display(img_in)
display(mask_in)

In [ ]:
output = mLG.pipe(
  prompt=prompt,
  negative_prompt="repetitive, distortion, glitch, borders, stretched, frames, breaks, multiple rows, gore, zombies, violence, splits, maps, diagrams, text, font, logos, branding",
  image=img_in,
  mask_image=mask_in,
  width=img_in.size[0], height=img_in.size[1],
  guidance_scale=12.0,
  num_inference_steps=32,
  num_images_per_prompt=1,
)
output.images[0]

In [ ]:
iw,ih = output.images[0].size
output.images[0].crop((256,0,iw,ih)).save("imgs/test01_grafted2.jpg")